In [2]:
! pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable


# **Importing Libraries**

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier  
from sklearn.linear_model import LogisticRegression, Ridge, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier 
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score


In [4]:
data = pd.read_csv("E:\Train_optima\Supply chain logisitcs problem.csv")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Turjo\AppData\Local\Temp\ipykernel_30316\114530144.py:1: SyntaxWarning: invalid escape sequence '\T'
  data = pd.read_csv("E:\Train_optima\Supply chain logisitcs problem.csv")


In [5]:
data

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,05/26/2013,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.300000
1,1.447158e+09,05/26/2013,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.940000
2,1.447139e+09,05/26/2013,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.200000
3,1.447364e+09,05/26/2013,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.160000
4,1.447364e+09,05/26/2013,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9210,1.447305e+09,05/26/2013,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9211,1.447319e+09,05/26/2013,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,339,2.354118
9212,1.447322e+09,05/26/2013,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683388,PLANT03,PORT09,245,0.294265
9213,1.447145e+09,05/26/2013,PORT04,V444_1,1,DTD,5,0,V55555555555555_8,1683430,PLANT03,PORT09,278,2.480000


In [6]:
print("First 5 rows of the dataset:")
print(data.head())

print("\nDataset Information:")
print(data.info())

print("\nSummary Statistics:")
print(data.describe(include='all'))

First 5 rows of the dataset:
       Order ID  Order Date Origin Port Carrier  TPT Service Level  \
0  1.447296e+09  05/26/2013      PORT09   V44_3    1           CRF   
1  1.447158e+09  05/26/2013      PORT09   V44_3    1           CRF   
2  1.447139e+09  05/26/2013      PORT09   V44_3    1           CRF   
3  1.447364e+09  05/26/2013      PORT09   V44_3    1           CRF   
4  1.447364e+09  05/26/2013      PORT09   V44_3    1           CRF   

   Ship ahead day count  Ship Late Day count   Customer  Product ID  \
0                     3                    0  V55555_53     1700106   
1                     3                    0  V55555_53     1700106   
2                     3                    0  V55555_53     1700106   
3                     3                    0  V55555_53     1700106   
4                     3                    0  V55555_53     1700106   

  Plant Code Destination Port  Unit quantity  Weight  
0    PLANT16           PORT09            808   14.30  
1    PLANT16 

# **Data Preprocessing**

In [7]:
data = data.fillna(data.median(numeric_only=True))  
data = data.fillna(data.mode().iloc[0])  

# Label Encoder Test

In [8]:
# categorical_columns = data.select_dtypes(include=['object']).columns
# label_encoders = {}
# for col in categorical_columns:
#     le = LabelEncoder()w
#     data[col] = le.fit_transform(data[col])
#     label_encoders[col] = le

# One Hot Encoding Test

In [9]:
categorical_columns = data.select_dtypes(include=['object']).columns
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')  # Replace sparse with sparse_output
encoded_categorical = onehot_encoder.fit_transform(data[categorical_columns])

encoded_categorical_df = pd.DataFrame(
    encoded_categorical,
    columns=onehot_encoder.get_feature_names_out(categorical_columns),
    index=data.index
)
data = pd.concat([data.drop(columns=categorical_columns), encoded_categorical_df], axis=1)


# **Model Pipeline**

In [10]:
target_column = 'TPT' 
X = data.drop(columns=[target_column])
y = data[target_column]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [12]:
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

C:\Users\Turjo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
model = RandomForestClassifier(random_state=42)  
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [15]:
y_pred = model.predict(X_test)

In [16]:
if len(y.unique()) > 2:
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
else:  
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       133
           1       1.00      1.00      1.00       637
           2       1.00      1.00      1.00      1940
           3       1.00      0.93      0.97        15
           4       0.88      0.90      0.89        40

    accuracy                           1.00      2765
   macro avg       0.97      0.96      0.96      2765
weighted avg       1.00      1.00      1.00      2765



# **Evaluation Matrix**

In [17]:

# results = {'Model': [], 'Accuracy': [], 'Precision': [], 'Recall': [], 'F1-Score': []}

# for name, model in models.items():
#     try:
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test)
#         accuracy = accuracy_score(y_test, y_pred)

#         report = classification_report(y_test, y_pred, output_dict=True)
#         precision = report['weighted avg']['precision']
#         recall = report['weighted avg']['recall']
#         f1_score = report['weighted avg']['f1-score']

#         results['Model'].append(name)
#         results['Accuracy'].append(accuracy)
#         results['Precision'].append(precision)
#         results['Recall'].append(recall)
#         results['F1-Score'].append(f1_score)
#     except Exception as e:
#         print(f"Error processing model {name}: {e}")

# for key, value in results.items():
#     print(f"{key}: {len(value)} entries")

# results_df = pd.DataFrame(results)

# print(results_df)


In [18]:
import joblib
joblib.dump(model, 'model.pkl')


['model.pkl']

In [19]:
import tkinter as tk
from tkinter import ttk
import joblib
import numpy as np

# Load the trained model (update the path to the actual model file if needed)
model_path = "model.pkl"  # Replace with actual path to your model
try:
    model = joblib.load(model_path)
except FileNotFoundError:
    print("Model file not found. Please ensure the model is saved as 'model.pkl'.")
    model = None

# Create the main Tkinter window
root = tk.Tk()
root.title("Supply Chain Logistics Prediction")

# Define input labels and fields based on dataset features
feature_labels = [
    "Order ID", "Order Date", "Origin Port", "Carrier", "Service Level",
    "Ship ahead day count", "Ship Late Day count", "Customer", "Product ID",
    "Plant Code", "Destination Port", "Unit quantity", "Weight"
]

# Create a dictionary to hold input values
input_values = {}

# Create input fields dynamically
for i, label in enumerate(feature_labels):
    ttk.Label(root, text=label).grid(row=i, column=0, padx=10, pady=5, sticky="w")
    entry = ttk.Entry(root, width=30)
    entry.grid(row=i, column=1, padx=10, pady=5)
    input_values[label] = entry

# Function to make predictions
def predict():
    if model is None:
        result_label.config(text="Model not loaded. Please check the model file.")
        return

    try:
        # Gather input values
        inputs = []
        for label in feature_labels:
            value = input_values[label].get()
            inputs.append(value)

        # Convert inputs to numpy array and reshape for prediction
        inputs = np.array(inputs, dtype=object).reshape(1, -1)

        # Make prediction
        prediction = model.predict(inputs)
        result_label.config(text=f"Predicted TPT: {prediction[0]}")

    except Exception as e:
        result_label.config(text=f"Error: {e}")

# Add a button to trigger prediction
predict_button = ttk.Button(root, text="Predict", command=predict)
predict_button.grid(row=len(feature_labels), column=0, columnspan=2, pady=10)

# Add a label to display the result
result_label = ttk.Label(root, text="", foreground="blue")
result_label.grid(row=len(feature_labels) + 1, column=0, columnspan=2, pady=10)

# Run the Tkinter main loop
root.mainloop()
